This is the tutorial for OOP in Julia. Note: in order to run this notebook, you will need to have Julia installed, which you can do [here](https://julialang.org/downloads/). If you don't want to deal with that right now, you will still be able to learn from reading through the code snippets throughout this notebook, but they will not be interactive.

First, we recreate our Qubit class:

In [2]:
mutable struct Qubit
    up::Float64
    down::Float64
end

Julia does not have methods associated to a class; instead, we ust define a function which takes a particular object as a parameter.

In [3]:
function X(self::Qubit)
    temp = self.up
    self.up = self.down
    self.down = temp
end

X (generic function with 1 method)

Finally, we'll use this code and confirm that our simple X gate works:

In [4]:
function main()
    q1 = Qubit(1, 0)
    X(q1)
    println(q1.up)
    println(q1.down)
end

main()

0.0
1.0


The token stream from our code can be generated easily using `Tokenize`. First, we install it

In [5]:
using Pkg
Pkg.add("Tokenize")

    Updating registry at `C:\Users\piegu\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\piegu\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\piegu\.julia\environments\v1.7\Manifest.toml`


And then, we use it like so.

In [6]:
using Tokenize

collect(tokenize("
mutable struct Qubit
    up::Float64
    down::Float64
end

function X(self::Qubit)
    temp = self.up
    self.up = self.down
    self.down = temp
end

function main()
    q1 = Qubit(1, 0)
    X(q1)
    println(q1.up)
    println(q1.down)
end"))

93-element Vector{Tokenize.Tokens.Token}:
 1,1-2,0          WHITESPACE     "\n"
 2,1-2,7          KEYWORD        "mutable"
 2,8-2,8          WHITESPACE     " "
 2,9-2,14         KEYWORD        "struct"
 2,15-2,15        WHITESPACE     " "
 2,16-2,20        IDENTIFIER     "Qubit"
 2,21-3,4         WHITESPACE     "\n    "
 3,5-3,6          IDENTIFIER     "up"
 3,7-3,8          OP             "::"
 3,9-3,15         IDENTIFIER     "Float64"
 3,16-4,4         WHITESPACE     "\n    "
 4,5-4,8          IDENTIFIER     "down"
 4,9-4,10         OP             "::"
 ⋮
 16,16-16,17      IDENTIFIER     "up"
 16,18-16,18      RPAREN         ")"
 16,19-17,4       WHITESPACE     "\n    "
 17,5-17,11       IDENTIFIER     "println"
 17,12-17,12      LPAREN         "("
 17,13-17,14      IDENTIFIER     "q1"
 17,15-17,15      OP             "."
 17,16-17,19      IDENTIFIER     "down"
 17,20-17,20      RPAREN         ")"
 17,21-18,0       WHITESPACE     "\n"
 18,1-18,3        KEYWORD        "end"
 18,4-18,3

While the output was too long, causing some tokens to be skipped over in the middle, this is still fairly representative. One thing to note is that this lexical analyzer chooses to include newline characters, with the token type of `WHITESPACE` which is fairly unusual. These are not usually considered meaningful tokens, or else they are tracked simply so that if an error arises, the compiler is able to report on which line it occured.

After we have our input tokenized, it is time for the AST to be generated. Julia once again has this easily accessible.

In [7]:
dump(:(
mutable struct Qubit
    up::Float64
    down::Float64
end,
function X(self::Qubit)
    temp = self.up
    self.up = self.down
    self.down = temp
end,
function main()
    q1 = Qubit(1, 0)
    X(q1)
    println(q1.up)
    println(q1.down)
end))

Expr
  head: Symbol tuple
  args: Array{Any}((3,))
    1: Expr
      head: Symbol struct
      args: Array{Any}((3,))
        1: Bool true
        2: Symbol Qubit
        3: Expr
          head: Symbol block
          args: Array{Any}((4,))
            1: LineNumberNode
              line: Int64 3
              file: Symbol In[7]
            2: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1: Symbol up
                2: Symbol Float64
            3: LineNumberNode
              line: Int64 4
              file: Symbol In[7]
            4: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1: Symbol down
                2: Symbol Float64
    2: Expr
      head: Symbol function
      args: Array{Any}((2,))
        1: Expr
          head: Symbol call
          args: Array{Any}((2,))
            1: Symbol X
            2: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1:

The entire AST is attached here for completeness:

``` Julia
Expr
  head: Symbol tuple
  args: Array{Any}((3,))
    1: Expr
      head: Symbol struct
      args: Array{Any}((3,))
        1: Bool true
        2: Symbol Qubit
        3: Expr
          head: Symbol block
          args: Array{Any}((4,))
            1: LineNumberNode
              line: Int64 3
              file: Symbol In[26]
            2: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1: Symbol up
                2: Symbol Float64
            3: LineNumberNode
              line: Int64 4
              file: Symbol In[26]
            4: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1: Symbol down
                2: Symbol Float64
    2: Expr
      head: Symbol function
      args: Array{Any}((2,))
        1: Expr
          head: Symbol call
          args: Array{Any}((2,))
            1: Symbol X
            2: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1: Symbol self
                2: Symbol Qubit
        2: Expr
          head: Symbol block
          args: Array{Any}((7,))
            1: LineNumberNode
              line: Int64 6
              file: Symbol In[26]
            2: LineNumberNode
              line: Int64 7
              file: Symbol In[26]
            3: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Symbol temp
                2: Expr
            4: LineNumberNode
              line: Int64 8
              file: Symbol In[26]
            5: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Expr
                2: Expr
            6: LineNumberNode
              line: Int64 9
              file: Symbol In[26]
            7: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Expr
                2: Symbol temp
    3: Expr
      head: Symbol function
      args: Array{Any}((2,))
        1: Expr
          head: Symbol call
          args: Array{Any}((1,))
            1: Symbol main
        2: Expr
          head: Symbol block
          args: Array{Any}((9,))
            1: LineNumberNode
              line: Int64 11
              file: Symbol In[26]
            2: LineNumberNode
              line: Int64 12
              file: Symbol In[26]
            3: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Symbol q1
                2: Expr
            4: LineNumberNode
              line: Int64 13
              file: Symbol In[26]
            5: Expr
              head: Symbol call
              args: Array{Any}((2,))
                1: Symbol X
                2: Symbol q1
            6: LineNumberNode
              line: Int64 14
              file: Symbol In[26]
            7: Expr
              head: Symbol call
              args: Array{Any}((2,))
                1: Symbol println
                2: Expr
            8: LineNumberNode
              line: Int64 15
              file: Symbol In[26]
            9: Expr
              head: Symbol call
              args: Array{Any}((2,))
                1: Symbol println
                2: Expr

```

As can be seen, the Julia AST is among the longer ones we've seen. It's > 100 lines for a fairly short code snippet. However, that is offset by how human-readable it is. It is much more minimal than clang's AST, for example; while that had many different types of nodes, each with many different attributes, Julia internally handles most functionality under the single `Expr` node. For example, both assignment operations and function calls are treated identically as `Expr` subtrees, where the `Head` value specifies the operation to be performed and the arguments (for function calls) or operands (for assignments) are stored as children. Even the struct definition is classified as an `Expr`. Finally, it is worth noting that this AST does indeed make use of all of the `\n` tokens we saw earlier, and a fair amount of the nodes are `LineNumberNode`s.

Finally, we want to see the LLVM IR for our Julia code. What if we try to just look at our `main()` function?

In [8]:
@code_llvm main()

;  @ In[4]:1 within `main`
; Function Attrs: uwtable
define void @julia_main_1989() #0 {
top:
;  @ In[4]:4 within `main`
  call void @j_println_1991(double 0.000000e+00) #0
;  @ In[4]:5 within `main`
  call void @j_println_1992(double 1.000000e+00) #0
  ret void
}


Julia has outsmarted us! We can see that ths LLVM is very optimized. The compiler has figured out that there are no conditionals, inputs, or calculations occuring; that our `Qubit` is not used for anything later on, so it's fields don't really matter; and that on every single run through the same result will occur. Accordingly, behind the scenes it has decided NOT to actually `X` our `Qubit`, but rather skip straight to the answer and simply print 1.0 and 0.0. While this is cool, we would like to see the LLVM for the entire program. Just like a high school math teacher, we will force the compiler to "show its work":

In [9]:
@code_llvm Qubit(1, 0)

;  @ In[5]:2 within `Qubit`
; Function Attrs: uwtable
define nonnull {}* @julia_Qubit_1649(i64 signext %0, i64 signext %1) #0 {
top:
  %2 = call {}*** inttoptr (i64 40556816 to {}*** ()*)() #3
; ┌ @ number.jl:7 within `convert`
; │┌ @ float.jl:146 within `Float64`
    %3 = sitofp i64 %0 to double
    %4 = sitofp i64 %1 to double
; └└
  %ptls_field3 = getelementptr inbounds {}**, {}*** %2, i64 2305843009213693954
  %5 = bitcast {}*** %ptls_field3 to i8**
  %ptls_load45 = load i8*, i8** %5, align 8
  %6 = call noalias nonnull {}* @jl_gc_pool_alloc(i8* %ptls_load45, i32 1416, i32 32) #4
  %7 = bitcast {}* %6 to i64*
  %8 = getelementptr inbounds i64, i64* %7, i64 -1
  store atomic i64 214847808, i64* %8 unordered, align 8
  %9 = bitcast {}* %6 to double*
  store double %3, double* %9, align 8
  %10 = bitcast {}* %6 to i8*
  %11 = getelementptr inbounds i8, i8* %10, i64 8
  %12 = bitcast i8* %11 to double*
  store double %4, double* %12, align 8
  ret {}* %6
}


There's the LLVM for the Qubit class. While we saw that its representation in the AST was much simpler than what clang generated for C++, we can see that behind the scenes it is much more similar, as it allocates space and plays with pointers in much the same way. 


Next, we'll do the same thing for the `X` gate:

In [10]:
@code_llvm X(Qubit(1, 0))

;  @ In[6]:1 within `X`
; Function Attrs: uwtable
define double @julia_X_1657({}* nonnull align 8 dereferenceable(16) %0) #0 {
top:
;  @ In[6]:2 within `X`
; ┌ @ Base.jl:42 within `getproperty`
   %1 = bitcast {}* %0 to double*
   %2 = load double, double* %1, align 8
; └
;  @ In[6]:3 within `X`
; ┌ @ Base.jl:42 within `getproperty`
   %3 = bitcast {}* %0 to i8*
   %4 = getelementptr inbounds i8, i8* %3, i64 8
   %5 = bitcast i8* %4 to double*
   %6 = load double, double* %5, align 8
; └
; ┌ @ Base.jl:43 within `setproperty!`
   store double %6, double* %1, align 8
; └
;  @ In[6]:4 within `X`
; ┌ @ Base.jl:43 within `setproperty!`
   store double %2, double* %5, align 8
; └
  ret double %2
}


The annotation Julia does on the LLVM makes it even friendlier and more human readable. The key takeaway is despite superficial differences in coding style and even AST structure, the LLVM representation of OOP in many different languages is robust and similar.